<a href="https://colab.research.google.com/github/juanmacaaz/Machine-Learning/blob/master/Tutorial_3_Clasificacion_de_Texto_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import tensorflow as tf

!pip install -q tensorflow-hub  # Libreria para reutilizacion de modulos de ML
!pip install -q tfds-nightly    # Libreria para coger datasets ya creados de manera mas comoda

import tensorflow_hub as hub
import tensorflow_datasets as tfds


print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")



Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is NOT AVAILABLE


In [2]:
# Cargamos los datos que vamos a utilizar

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0RIWM1/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0RIWM1/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete0RIWM1/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

# Formato de salida es
# train_labels_batch --> <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>
# train_data         --> <tf.Tensor: shape=(10,), dtype=string, numpy=[b'I have been known to fall asleep during films, but this is us]....

In [4]:
# Para este tipo de problemas (tratar texto) nos preguntamos varias cosas
# Como represantar el texto?
# Cuantas capas usar en el modelo?
# Cuantas unidades ocultas (neuronas) usar en cada capa?

# Una forma eficiente de solucionar este problema (tratar texto de forma computacional) es utilizando una tecnica de "embed" el texto a vectores
# La forma de hacer esto es mediante una red neuroal (Que la cogeremos del TF Hub)

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1" # Ahi cogeremos el modelo
hub_layer = hub.KerasLayer(embedding, input_shape=[],                   # Creamos la capa
                           dtype=tf.string, trainable=True)

hub_layer(train_examples_batch)                                         # Transforma de texto a vextores


<tf.Tensor: id=378, shape=(10, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064],
       [ 1.5165    , -0.71034056,  1.8556767 , -1.2033532 , -1.

In [6]:
# Ahora creamos el modelos de TF

model = tf.keras.Sequential() # Forma en la que definimos el modelo (Sequencial)

model.add(hub_layer)          # Usamos la capara de entrada de hub_layer para pasar las palabras a vectores
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compilamos el modelo

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [10]:
# Entrenamos la red neuronal con 100000 elementos en batches de 512

history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)


Epoch 1/20
30/30 [==============================] - 5s 168ms/step - loss: 0.7049 - accuracy: 0.5569 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 4s 149ms/step - loss: 0.6393 - accuracy: 0.6510 - val_loss: 0.6182 - val_accuracy: 0.6704
Epoch 3/20
30/30 [==============================] - 4s 144ms/step - loss: 0.5987 - accuracy: 0.6860 - val_loss: 0.5830 - val_accuracy: 0.6989
Epoch 4/20
30/30 [==============================] - 4s 148ms/step - loss: 0.5603 - accuracy: 0.7162 - val_loss: 0.5480 - val_accuracy: 0.7281
Epoch 5/20
30/30 [==============================] - 5s 162ms/step - loss: 0.5174 - accuracy: 0.7535 - val_loss: 0.5072 - val_accuracy: 0.7592
Epoch 6/20
30/30 [==============================] - 5s 161ms/step - loss: 0.4757 - accuracy: 0.7868 - val_loss: 0.4728 - val_accuracy: 0.7852
Epoch 7/20
30/30 [==============================] - 5s 151ms/step - loss: 0.4349 - accuracy: 0.8148 - val_loss: 0.4389 - val_accuracy: 0.804

In [12]:
# Evaluamos el modelo con los datos de test_data

results = model.evaluate(test_data.batch(512), verbose=2)
# results = [0.32221849171482786, 0.8646]
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))


49/49 - 3s - loss: 0.3222 - accuracy: 0.8646
loss: 0.322
accuracy: 0.865


In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.
